# PREPARE DATA

Load and concatenate dataframes from individual months

## Prepare

In [1]:
import pandas as pd

In [2]:
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

In [3]:
DROPNANS = True
INCIDENT = 'roubo_celular'

BASE_COLS = ['DATAOCORRENCIA', 'HORAOCORRENCIA', 'PERIDOOCORRENCIA',]

# Neighborhood dataframe
COLS_HOOD = BASE_COLS + ['BAIRRO']

# Localizations dataframe
COLS_LOC = BASE_COLS + ['LATITUDE', 'LONGITUDE']

# Prints to check everything is ok
DEBUG_MODE = True

In [4]:
#periods = [f'2023_{i:02d}' for i in range(1, 13)] + ['2024_01']
periods = [f'2023_{i:02d}' for i in range(6, 13)] + ['2024_01']
periods

['2023_06',
 '2023_07',
 '2023_08',
 '2023_09',
 '2023_10',
 '2023_11',
 '2023_12',
 '2024_01']

In [5]:
# just for reference
dfx = pd.read_excel(f"../data/{INCIDENT}/2024_01.xlsx")

In [6]:
dfx.columns

Index(['ANO_BO', 'NUM_BO', 'NUMERO_BOLETIM', 'BO_INICIADO', 'BO_EMITIDO',
       'DATAOCORRENCIA', 'HORAOCORRENCIA', 'PERIDOOCORRENCIA',
       'DATACOMUNICACAO', 'DATAELABORACAO', 'BO_AUTORIA', 'FLAGRANTE',
       'NUMERO_BOLETIM_PRINCIPAL', 'LOGRADOURO', 'NUMERO', 'BAIRRO', 'CIDADE',
       'UF', 'LATITUDE', 'LONGITUDE', 'DESCRICAOLOCAL', 'EXAME', 'SOLUCAO',
       'DELEGACIA_NOME', 'DELEGACIA_CIRCUNSCRICAO', 'ESPECIE', 'RUBRICA',
       'DESDOBRAMENTO', 'STATUS', 'TIPOPESSOA', 'VITIMAFATAL', 'NATURALIDADE',
       'NACIONALIDADE', 'SEXO', 'DATANASCIMENTO', 'IDADE', 'ESTADOCIVIL',
       'PROFISSAO', 'GRAUINSTRUCAO', 'CORCUTIS', 'NATUREZAVINCULADA',
       'TIPOVINCULO', 'RELACIONAMENTO', 'PARENTESCO', 'PLACA_VEICULO',
       'UF_VEICULO', 'CIDADE_VEICULO', 'DESCR_COR_VEICULO',
       'DESCR_MARCA_VEICULO', 'ANO_FABRICACAO', 'ANO_MODELO',
       'DESCR_TIPO_VEICULO', 'QUANT_CELULAR', 'MARCA_CELULAR'],
      dtype='object')

In [ ]:
df_hood = None
df_loc = None
for period in periods:
    df = pd.read_excel(f"../data/{INCIDENT}/{period}.xlsx")
    #df['DATAOCORRENCIA'] = pd.to_datetime(df['DATAOCORRENCIA'], format='%d/%m/%Y')
    df[COL_DATETIME] = pd.to_datetime(df[COL_DATETIME]) #, format='%d/%m/%Y')
    if DEBUG_MODE:
        print(
            f"{period}\t{df['DATAOCORRENCIA'].min()}\t{df['DATAOCORRENCIA'].max()}"
        )
    df_hood = pd.concat([df_hood, df[COLS_HOOD]])
    df_loc = pd.concat([df_loc, df[COLS_LOC]])

2023_06	2013-07-06 00:00:00	2023-12-06 00:00:00
2023_07	2012-03-07 00:00:00	2023-12-07 00:00:00
2023_08	2004-10-08 00:00:00	2023-12-08 00:00:00
2023_09	2004-11-17 00:00:00	2023-12-09 00:00:00
2023_10	2004-10-21 00:00:00	2023-12-10 00:00:00
2023_11	2013-02-11 00:00:00	2023-12-11 00:00:00
2023_12	2005-11-28 00:00:00	2023-12-31 00:00:00
2024_01	2012-12-31 00:00:00	2024-12-01 00:00:00


In [8]:
print(df_hood.shape, df_loc.shape)
df_hood = df_hood.dropna(subset=["BAIRRO"])
df_loc = df_loc.dropna(subset=["LONGITUDE", "LATITUDE"])
print(df_hood.shape, df_loc.shape)

## TODO FIX KeyError: ['LONGITUDE', 'LATITUDE']
## for neighborhoods we can also drop localization NaNs
#if DROPNANS:
#    df_hood = df_hood.dropna(subset=["LONGITUDE", "LATITUDE"])
#print(df_hood.shape)

(164931, 4) (164931, 5)
(161766, 4) (143999, 5)


In [9]:
df_hood["BAIRRO"].value_counts()

BAIRRO
CENTRO                3094
Centro                2561
CAPAO REDONDO         1880
REPUBLICA             1563
ITAIM BIBI            1349
                      ... 
Quinta do Guarau         1
ROTATÓRIA                1
Vila Santa Helena        1
Residencial Aimoré       1
PORTAL DOS NOBRES        1
Name: count, Length: 8842, dtype: int64

In [10]:
dfx['DATAOCORRENCIA'] = pd.to_datetime(dfx['DATAOCORRENCIA'], format='%d/%m/%Y')

In [11]:
# all ok